# Lab 6 - PySpark DataFrames.

# 0. Uruchomienie silnika Spark.

In [1]:
# dla instalacji lokalnej

import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
os.environ['PYSPARK_HOME'] = sys.executable

In [ ]:
# dla instalacji z dockerem

import os
os.environ['SPARK_NAME'] = "/opt/spark"
# os.environ['PYSPARK_DRIVER_PYTHON'] = 'jupyter'
os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = 'lab'
# os.environ['PYSPARK_PYTHON'] = 'python'
os.environ['PYSPARK_DRIVER_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'
os.environ['PYSPARK_PYTHON'] = '/opt/spark/work-dir/.venv/bin/python3'

# można też spróbować wykorzystać moduł findspark do automatycznego odnalezienia miejsca instalacji sparka
# import findspark
# findspark.init()
# lub
# findspark.init("/opt/spark")

In [2]:
from pyspark.sql import SparkSession

# spark = SparkSession.builder.master("spark://spark-master:7077").appName("Create-DataFrame").getOrCreate()
# konfiguracja z określeniem liczby wątków (2) oraz ilości pamięci do wykorzystania poza stertą interpretera Pythona
spark = SparkSession\
        .builder\
        .master("local[2]")\
        .appName("Create-DataFrame")\
        .config("spark.memory.offHeap.enabled","true")\
        .config("spark.memory.offHeap.size","6g")\
        .getOrCreate()

In [3]:
spark.sparkContext

<SparkContext master=local[2] appName=Create-DataFrame>

In [9]:
sc = spark.sparkContext

# 1. Spark DataFrame.

> Spark Dataframe API: https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.DataFrame.html

Zanim przejdziemy do obiektów typu DataFrame warto powiedzieć, że w systemie Spark występuje również tym Dataset, co może prowadzić do używania tych dwóch terminów zamiennie, co byłoby błędem. Obiekty typu Dataset są odrębnym typem i póki co nie są one dostępne w API Pythona dla Sparka, ale można na nich pracować z poziomu API Javy oraz języka Scala.
Kilka szczegółów na temat tego typu oraz jego tworzenia z poziomu języka Java lub Scala można znaleźć [tu](https://spark.apache.org/docs/3.5.3/sql-getting-started.html#creating-datasets) oraz [tu](https://spark.apache.org/docs/3.5.3/api/java/index.html).
Obiekty typu Dataset w języku Java i Scala są obiektami silnie typowanymi, więc mamy do dyspozycji transfomacje typowane, a w Pythonie są to transformacje nietypowane (z racji natury języka Python).

Spark DataFrame to rozproszona kolekcja danych Spark do pracy z danymi ustrukturyzowanymi, która podobna jest do obiektów DataFrame znanych z biblioteki pandas oraz języka R jednak dużo bardziej zoptymalizowana w kontekście pracy w środowisku rozproszonym. 


## Pobranie danych i wczytanie do ramki Spark.

In [38]:
# pobranie spakowanego zbioru za pomocą polecenia systemowego wget
# strona datasetu: https://archive.ics.uci.edu/dataset/911/recipe+reviews+and+user+feedback+dataset
# to zadziała tylko w systemach Linux, w Windows można użyć polecenia curl
# !wget https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

# Windows
!curl -O https://archive.ics.uci.edu/static/public/911/recipe+reviews+and+user+feedback+dataset.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 61365    0 61365    0     0  44491      0 --:--:--  0:00:01 --:--:-- 44629
100  835k    0  835k    0     0   349k      0 --:--:--  0:00:02 --:--:--  349k
100 1907k    0 1907k    0     0   565k      0 --:--:--  0:00:03 --:--:--  565k
100 2064k    0 2064k    0     0   609k      0 --:--:--  0:00:03 --:--:--  610k


In [39]:
# listujemy zawartość bieżącego folderu (również Linux, ale jak mamy WSL to i Windows)
!ls

# Windows
# !dir

01_pyspark_introduction.ipynb
01_pyspark_introduction_with_solutions.ipynb
02_pyspark_dataframes.ipynb
clean_words.json
data
recipe+reviews+and+user+feedback+dataset.zip


In [40]:
# zmiana nazwy pliku - nie jest konieczna, ale trzeba zmienić później ścieżkę w kolejnej komórce notatnika
# !mv recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

# Windows
!ren recipe+reviews+and+user+feedback+dataset.zip recipe_reviews.zip

In [41]:
# wypakowujemy plik do podfolderu data
import zipfile
with zipfile.ZipFile("recipe_reviews.zip", 'r') as zip_ref:
    zip_ref.extractall("./data")

In [42]:
!ls ./data

# lub
# !dir ./data

pan-tadeusz.txt
polish.stopwords.txt
Recipe Reviews and User Feedback Dataset.csv


In [45]:
# sprawdzamy jak wyglądają 3 pierwsze linie pliku, widać, że pierwsza zawiera nagłówki kolumn a dane są oddzielone przecinkiem
# Linux, mac i WSL
!head -3 "data/Recipe Reviews and User Feedback Dataset.csv"
# 3 ostatnie linie z pliku
# !tail -3 "data/Recipe Reviews and User Feedback Dataset.csv"

,recipe_number,recipe_code,recipe_name,comment_id,user_id,user_name,user_reputation,created_at,reply_count,thumbs_up,thumbs_down,stars,best_score,text
0,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2G3aneMRgRMZwXqIHmSdXSG1hEM,u_9iFLIhMa8QaG,Jeri326,1,1665619889,0,0,0,5,527,"I tweaked it a little, removed onions because of onion haters in my house, used Italian seasoning instead of just oregano, and use a paprika/ cayenne mix and a little more than the recipe called for.. we like everything a bit more hot. The chili was amazing! It was easy to make and everyone absolutely loved it. It will now be a staple meal in our house."
1,001,14299,Creamy White Chili,sp_aUSaElGf_14299_c_2FsPC83HtzCsQAtOxlbL6RcaPbY,u_Lu6p25tmE77j,Mark467,50,1665277687,0,7,0,5,724,"Bush used to have a white chili bean and it made this recipe super simple. Iâ€™ve written to them and asked them to please!, bring them back"


In [46]:
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",")

## Wyświetlenie danych oraz schematu

In [64]:
# najpopularniejsza metoda ich pobrania to show(), ale jest ich więcej
df_reviews.show(5)

+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id| user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|   Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|   Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush used to have...|
|  2|          

In [49]:
# rzut oka na schemę tego DataFrame
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: string (nullable = true)
 |-- reply_count: string (nullable = true)
 |-- thumbs_up: string (nullable = true)
 |-- thumbs_down: string (nullable = true)
 |-- stars: string (nullable = true)
 |-- best_score: string (nullable = true)
 |-- text: string (nullable = true)



In [50]:
# widać, że wszystkie kolumny są typu string, to jest domyślny sposób wczytywania danych przez spark z plain text
# możemy jednak przekazać dodatkowy parametr, który na podstawie próbki danych spróbuje dobrać typ danych odpowiedni dla kolumny
df_reviews = spark.read.csv('./data/Recipe Reviews and User Feedback Dataset.csv', header=True, sep=",", inferSchema=True)

In [51]:
# po wypisaniu schemy widać zmianę
df_reviews.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- recipe_number: string (nullable = true)
 |-- recipe_code: string (nullable = true)
 |-- recipe_name: string (nullable = true)
 |-- comment_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- user_reputation: string (nullable = true)
 |-- created_at: integer (nullable = true)
 |-- reply_count: integer (nullable = true)
 |-- thumbs_up: integer (nullable = true)
 |-- thumbs_down: integer (nullable = true)
 |-- stars: integer (nullable = true)
 |-- best_score: integer (nullable = true)
 |-- text: string (nullable = true)



> Listę dostępnych typów danych znajdziesz tu: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql/data_types.html

In [54]:
# ramkę możemy również inicjalizować wskazując pożądane typy danych
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DecimalType, LongType

data = [("James","","Smith",36636,"M",3000),
    ("Michael","Rose","",40288,"M",4000),
    ("Robert","","Williams",42114,"M",4000),
    ("Maria","Anne","Jones",39192,"F",4000),
    ("Jen","Mary","Brown","","F",1000)
  ]

schema = StructType([ \
    StructField("firstname", StringType(), True), \
    StructField("user_id", StringType(), True), \
    StructField("lastname", StringType(), True), \
    StructField("id", StringType(), True), \
    # błąd konwersji "" na int!
    # StructField("id", LongType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", StringType(), True)
    # chcielibyśmy tak, ale tutaj nie da się za bardzo - błąd konwersji int na decimal!
    # StructField("salary", DecimalType(10,2), True) \
  ])

df_test = spark.createDataFrame(data=data,schema=schema)
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: string (nullable = true)



In [55]:
# możemy wykonać rzutowanie po wczytaniu danych z większością kolumn typu tekstowego
import pyspark.sql.functions as F

df_test = df_test.withColumn("salary", F.col("salary").cast("decimal(10,2)"))
df_test.printSchema()

root
 |-- firstname: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: decimal(10,2) (nullable = true)



In [56]:
# wyświetlenie danych z pojedynczej kolumny
df_test.select(df_test.salary).show()

+-------+
| salary|
+-------+
|3000.00|
|4000.00|
|4000.00|
|4000.00|
|1000.00|
+-------+



In [57]:
# ile wierszy w ramce?
df_reviews.count()

18268

In [58]:
# DataFrame składa się z obiektów typu Column dla każdej kolumny
# API dla typu Column: https://spark.apache.org/docs/3.5.3/api/python/reference/pyspark.sql/api/pyspark.sql.Column.html

# do kolumn możemy się odwoływać tak jak w pandas API, ale wynik jest inny
df_reviews.user_name, df_reviews['user_name']

(Column<'user_name'>, Column<'user_name'>)

In [59]:
# aby wyświetlić dane musimy wywoałać funkcję select na obiekcie dataframe

df_reviews.select(df_reviews.user_name).show(5)

+----------+
| user_name|
+----------+
|   Jeri326|
|   Mark467|
|Barbara566|
|jeansch123|
|  camper77|
+----------+
only showing top 5 rows



In [60]:
# do funkcji select możemy przekazać wiele kolumn a wywołania podobnie jak dla RDD są leniwe
print(df_reviews.select(df_reviews.user_name, df_reviews.user_reputation))
# musimy więc wywołać funkcję, której wykonanie "zmusi" Sparka do wyliczenia jej wartości lub jawnie wywołać np. show
df_reviews.select(df_reviews.user_name, df_reviews.user_reputation).show(5)

DataFrame[user_name: string, user_reputation: string]
+----------+---------------+
| user_name|user_reputation|
+----------+---------------+
|   Jeri326|              1|
|   Mark467|             50|
|Barbara566|             10|
|jeansch123|              1|
|  camper77|             10|
+----------+---------------+
only showing top 5 rows



In [ ]:
# można zmienić to domyślne zachowanie Spark, ale zazwyczaj nie jest to dobry pomysł, chyba, że zbiór jest mały
# zmieniamy to poprzez edycję poniższego parametru
# spark.conf.set('spark.sql.repl.eagerEval.enabled', True)

In [61]:
# lub indeksując kolumny innym sposobem
df_reviews.select(df_reviews['user_name'],df_reviews['user_reputation']).show(5)

+----------+---------------+
| user_name|user_reputation|
+----------+---------------+
|   Jeri326|              1|
|   Mark467|             50|
|Barbara566|             10|
|jeansch123|              1|
|  camper77|             10|
+----------+---------------+
only showing top 5 rows



In [62]:
from pyspark.sql.functions import isnan, when, count, col

# policzymy teraz liczbę wartości NULL w każdej kolumnie
df_reviews.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews.columns]).show()

+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|_c0|recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|  0|           45|         64|         74|        77|     80|       83|             84|        86|         86|       86|         86|   86|        86|  86|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+



## Filtrowanie danych

In [65]:
# rzućmy okiem na kilka wierszy gdzie w kolumnie recipe_name jest wartość NULL
df_reviews.filter(df_reviews.recipe_code.isNull()).show()

+--------------------+--------------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|                 _c0|       recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+--------------------+--------------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|      Thank you!!!!"|                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|           NULL|      NULL|       NULL|     NULL|       NULL| NULL|      NULL|NULL|
| It was excellent!  |                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|           NULL|      NULL|       NULL|     NULL|       NULL| NULL|      NULL|NULL|
|The recipe was a ...|                NULL|       NULL|       NULL|      NULL|   NULL|     NULL|    

In [66]:
# zapisanie do nowej ramki danych bez wartości pustych
df_reviews_clean = df_reviews.na.drop()
df_reviews_clean.count()

18182

In [67]:
# dla pewności możemy to sprawdzić raz jeszcze
df_reviews_clean.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_reviews_clean.columns]).show()

+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|_c0|recipe_number|recipe_code|recipe_name|comment_id|user_id|user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|text|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+
|  0|            0|          0|          0|         0|      0|        0|              0|         0|          0|        0|          0|    0|         0|   0|
+---+-------------+-----------+-----------+----------+-------+---------+---------------+----------+-----------+---------+-----------+-----+----------+----+



In [69]:
# filtrowanie danych z ramki
df_reviews_clean.filter(df_reviews.user_name.startswith('a')).select(df_reviews_clean.user_name).show(10)
df_reviews_clean.filter(df_reviews.stars == 5).show(10)

+---+-------------+-----------+------------------+--------------------+--------------+----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|_c0|recipe_number|recipe_code|       recipe_name|          comment_id|       user_id|       user_name|user_reputation|created_at|reply_count|thumbs_up|thumbs_down|stars|best_score|                text|
+---+-------------+-----------+------------------+--------------------+--------------+----------------+---------------+----------+-----------+---------+-----------+-----+----------+--------------------+
|  0|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_9iFLIhMa8QaG|         Jeri326|              1|1665619889|          0|        0|          0|    5|       527|I tweaked it a li...|
|  1|          001|      14299|Creamy White Chili|sp_aUSaElGf_14299...|u_Lu6p25tmE77j|         Mark467|             50|1665277687|          0|        7|          0|    5|       724|Bush us

In [70]:
from pyspark.sql.functions import avg

# wyliczenie średniej wartości z kolumny
df_reviews_clean.select(avg(df_reviews_clean.thumbs_up)).show()

+------------------+
|    avg(thumbs_up)|
+------------------+
|1.0892641073589264|
+------------------+



In [71]:
# ale możemy się dowiedzieć tego i więcej w sposób podobny do tego z biblioteki pandas
df_reviews_clean.select(df_reviews_clean.thumbs_up).describe().show()

+-------+------------------+
|summary|         thumbs_up|
+-------+------------------+
|  count|             18182|
|   mean|1.0892641073589264|
| stddev| 4.201003572820717|
|    min|                 0|
|    max|               106|
+-------+------------------+



In [72]:
from pyspark.sql.functions import desc

df_reviews_clean.groupby('recipe_code').agg({'thumbs_down': 'sum'}).sort(desc('sum(thumbs_down)')).show(10)

+-----------+----------------+
|recipe_code|sum(thumbs_down)|
+-----------+----------------+
|       2832|             488|
|       9739|             354|
|      17826|             328|
|      18345|             313|
|      12003|             306|
|       4383|             301|
|      41095|             275|
|       8202|             272|
|       6504|             264|
|       6086|             262|
+-----------+----------------+
only showing top 10 rows



Dla potrzeb laboratorium została stworzona funkcja, która pozwoli na generowanie datasetu i zapisane go w pliku csv na początek, aby zaprezentować podstawowe typy danych Sparka.

In [73]:
# deklaracja zbiorów wartości dla poszczególnych kolumn przyszłego zbioru danych
header = ['id', 'firstname', 'lastname', 'age', 'salary']
firstnames = ['Adam', 'Katarzyna', 'Krzysztof', 'Marek', 'Aleksandra', 'Zbigniew', 'Wojciech', 'Mieczysław', 'Agata', 'Wisława']
lastnames = ['Mieczykowski', 'Kowalski', 'Malinowski' , 'Szczaw', 'Glut', 'Barański', 'Brzęczyszczykiewicz', 'Wróblewski', 'Wlotka', 'Pysla']
age = {'min': 18, 'max': 68}
salary = {'min': 3200, 'max': 12500}

In [ ]:
!pip install tqdm

In [76]:
# funkcja do generowania fikcyjnego datasetu
# n_rows oznacza ilość wierszy, którą chcemy finalnie uzyskać


import random
from tqdm import tqdm

def build_dataset(filename, n_rows=100, chunk_size=100000):
    rows = []
    rows.append(header)
    mu = (salary['max'] + salary['min']) / 2
    sigma = 1000

    with open(filename, 'w', encoding='utf-8') as filehandler:
        
        for id in tqdm(range(1, n_rows + 1), total=n_rows, desc="Building dataset..."):
            row = [
                f'{id}', 
                f'{random.choice(firstnames)}', 
                f'{random.choice(lastnames)}', 
                f"{random.randint(age['min'], age['max'])}",
                f"{round(float(random.normalvariate(mu=mu, sigma=sigma)), 2)}"
            ]
            rows.append(row)
            if id % chunk_size == 0:
                filehandler.writelines([f"{','.join(row)}\n" for row in rows])
                rows = []


In [77]:
# około 750MB zostanie zapisanych w pliku csv, dostosuj ilość rekordów do swoich potrzeb
build_dataset('./data/employee.csv', 20_000_000)
# build_dataset('./data/employee_100m.csv', 100_000_000)

Building dataset...: 100%|███████████████████████████| 20000000/20000000 [01:43<00:00, 192591.80it/s]


In [4]:
%%time
# więcej magicznych metod w Jupyter Notebooku: https://ipython.readthedocs.io/en/stable/interactive/magics.html
# wczytanie pliku csv przez spark
# df = spark.read.csv('employee.csv', header=True)
df = spark.read.csv('./data/employee.csv', header=True, inferSchema=True)

CPU times: total: 0 ns
Wall time: 18.2 s


In [6]:
type(df)

pyspark.sql.dataframe.DataFrame

In [81]:
%%time
# wypisujemy schemat i 10 pierwszych wierszy utworzonego obiektu Spark DataFrame
df.printSchema()
df.show(10)

root
 |-- id: integer (nullable = true)
 |-- firstname: string (nullable = true)
 |-- lastname: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- salary: double (nullable = true)

+---+----------+-------------------+---+--------+
| id| firstname|           lastname|age|  salary|
+---+----------+-------------------+---+--------+
|  1|  Zbigniew|           Barański| 46| 8797.82|
|  2| Krzysztof|           Kowalski| 33| 7441.71|
|  3| Krzysztof|Brzęczyszczykiewicz| 60| 8502.79|
|  4|      Adam|         Wróblewski| 36|10258.55|
|  5|   Wisława|           Barański| 43|  9006.9|
|  6|Aleksandra|         Wróblewski| 38| 8796.75|
|  7|     Agata|               Glut| 64| 9252.93|
|  8| Krzysztof|             Wlotka| 58| 8470.38|
|  9|Mieczysław|              Pysla| 51|10216.48|
| 10| Krzysztof|Brzęczyszczykiewicz| 48| 7853.63|
+---+----------+-------------------+---+--------+
only showing top 10 rows

CPU times: total: 0 ns
Wall time: 79.7 ms


In [5]:
# przykład wykorzystania funkcji transform, która mapuje wykonanie stworzonej funkcji tu_upper_str_columns na istniejącą kolumnę
# i zwraca nową ramkę z dodatkową kolumną
from pyspark.sql.functions import upper

def to_upper_str_columns(df, column_name, new_column_name):
    return df.withColumn(new_column_name, upper(df[column_name]))

df = df.transform(to_upper_str_columns, "firstname", "firstname_upper")

In [8]:
df.show(10)

+---+----------+-------------------+---+--------+---------------+
| id| firstname|           lastname|age|  salary|firstname_upper|
+---+----------+-------------------+---+--------+---------------+
|  1|  Zbigniew|           Barański| 46| 8797.82|       ZBIGNIEW|
|  2| Krzysztof|           Kowalski| 33| 7441.71|      KRZYSZTOF|
|  3| Krzysztof|Brzęczyszczykiewicz| 60| 8502.79|      KRZYSZTOF|
|  4|      Adam|         Wróblewski| 36|10258.55|           ADAM|
|  5|   Wisława|           Barański| 43|  9006.9|        WISŁAWA|
|  6|Aleksandra|         Wróblewski| 38| 8796.75|     ALEKSANDRA|
|  7|     Agata|               Glut| 64| 9252.93|          AGATA|
|  8| Krzysztof|             Wlotka| 58| 8470.38|      KRZYSZTOF|
|  9|Mieczysław|              Pysla| 51|10216.48|     MIECZYSŁAW|
| 10| Krzysztof|Brzęczyszczykiewicz| 48| 7853.63|      KRZYSZTOF|
+---+----------+-------------------+---+--------+---------------+
only showing top 10 rows



In [84]:
# filtrowanie numeryczne, ale tu na kolumnie typu str - czy jest poprawne?
df.filter(df["salary"] > 10000).count()

314855

In [85]:
# na ile partycji została nasza ramka danych rozrzucona po "klastrze"?
df.rdd.getNumPartitions()

6

In [86]:
%%time
# mierzymy czas operacji przy domyślnej liczbie partycji
df.filter(df["salary"] > 10000).count()

CPU times: total: 0 ns
Wall time: 13.1 s


314855

In [12]:
df = df.repartition(12)

In [13]:
df.rdd.getNumPartitions()

12

In [89]:
%%time
# mierzymy czas operacji przy 12 partycjach dla 20_000_000 rekordów
df.filter(df["salary"] > 10000).count()

CPU times: total: 0 ns
Wall time: 13.8 s


314855

In [ ]:
# macierz częstości dla dwóch kolumn - uwaga dla bardzo różnorodnych danych!
df.crosstab("firstname", "age").sort("firstname_age").show()

In [91]:
# funkcja explain może przydać się w przypadku bardziej zaawansowanego debuggingu, optymalizacji i zrozumienia
# kolejności działania niektórych elementów silnika Spark
query = df.filter(df.firstname.contains('ski'))
query.explain(mode='formatted')

== Physical Plan ==
AdaptiveSparkPlan (5)
+- Exchange (4)
   +- Project (3)
      +- Filter (2)
         +- Scan csv  (1)


(1) Scan csv 
Output [5]: [id#1219, firstname#1220, lastname#1221, age#1222, salary#1223]
Batched: false
Location: InMemoryFileIndex [file:/C:/Users/Krzysztof/__projects/__pyspark_local_aiids/data/employee.csv]
PushedFilters: [IsNotNull(firstname), StringContains(firstname,ski)]
ReadSchema: struct<id:int,firstname:string,lastname:string,age:int,salary:double>

(2) Filter
Input [5]: [id#1219, firstname#1220, lastname#1221, age#1222, salary#1223]
Condition : (isnotnull(firstname#1220) AND Contains(firstname#1220, ski))

(3) Project
Output [6]: [id#1219, firstname#1220, lastname#1221, age#1222, salary#1223, upper(firstname#1220) AS firstname_upper#1256]
Input [5]: [id#1219, firstname#1220, lastname#1221, age#1222, salary#1223]

(4) Exchange
Input [6]: [id#1219, firstname#1220, lastname#1221, age#1222, salary#1223, firstname_upper#1256]
Arguments: RoundRobinPartitioni

In [6]:
# jeżeli przy zapisie w systemie Windows pojawia się błąd, to jest to związane z
# brakiem lub niewłaściwym sposobem konfiguracji winutils:
# https://github.com/steveloughran/winutils

# krok 1 - pobieramy całą zawartość folderu dla naszej wersji Hadoop - tu mamy wersję 3.0.0 
# (możemy sklonować całe repozytorium poleceniem git clone https://github.com/steveloughran/winutils lub
# pobrać je jako plik zip bezpośrednio ze strony GitHub)
# krok 2 - tworzymy folder, i mapujemy go na zmienną środowiskową HADOOP_HOME
# krok 3 - kopiujemy folder bin z pobranego folderu HADOOP-3.0.0 (cały folder) do folderu z pkt. 2
# krok 4 - dodajemy ścieżkę folder z kroku 2/bin do zmiennej środowiskowej path

# krok 5 - przeładowujemy zmienne środowiskowe co zonacza konieczność zamknięcia np. VSC oraz przeładowania
# jądra Python i uruchomienia sesji Spark od nowa

# sprawdzamy jaka jest wartość zmiennej środowiskowej HADOOP_HOME
!echo %hadoop_home%

C:\hadoop


In [ ]:
# zapisujemy ramkę do plików parquet
# zwróć uwagę na liczbę utworzonych plików

df.write.parquet('./data/parquet/')

In [7]:
# lub chcąc nadpisać już istniejące dane - w trybie overwrite
df.write.mode("overwrite").parquet('./data/parquet/')

In [10]:
sc.stop()

### Zadania

**Zadanie 1**  
Na zbiorze danych '_Recipe Reviews ..._' wykonaj:  
1.1  Zmień nazwę pierwszej kolumny z `_c0` na `id`.  
1.2  Wyświetl 10 najwyższych wartości w kolumnie `reply_count`.  
1.3  Wyświetl 10 najwyższych sum wartości w kolumnie `best_score` dla każdego przepisu (grupowanie).  
1.4  Które 10 przepisów miało najwięcej komentarzy?  
1.5  Wyświetl rozkład wartości w kolumnie `stars`.  


**Zadanie 2**  
Wczytaj zbiór danych `employee` nakazując Sparkowi wywnioskowanie bardziej optymalnych typów danych niż domyślny typ `string`.

**Zadanie 3**  
Jaki jest czas wykonania operacji `df.filter(df["salary"] > 10000).count()` tym razem przy numerycznym typie kolumny `salary`? Jest jakaś różnica?

**Zadanie 4**  
Wykorzystując przykład z dokumentacji klasy `Bucketizer` (https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.Bucketizer.html) podziel dane w kolumnie `age` zbioru `employee` na buckety co 10 lat (10-19, 20-29, ..., 60-69) i wyświetl te dane dla 20 pierwszych wierzy w formie surowej oraz całość grupując po bucketach i licząc ile osób znalazło się w każdym z nich.